In [77]:
from geopy.distance import geodesic
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import time

In [204]:
#df = pd.read_csv('capitals.csv', skiprows=[2,9])
df = pd.read_csv('capitals.csv')

In [38]:
def generate_random_triplet(n):
    head = np.random.randint(0,n)
    
    left = np.random.randint(0,n)
    while left == head:
        left = np.random.randint(0,n)
        
    right = np.random.randint(0,n)
    while right == left or right == head:
        right = np.random.randint(0,n)
        
    return [head, left, right]

In [73]:
triplet = generate_random_triplet(len(df))
print(df['name'][triplet[0]])
print(df['name'][triplet[1]])
print(df['name'][triplet[2]])
print(triplet)
get_comparison(triplet)

Montana
Michigan
Iowa
[23, 19, 12]


False

In [63]:
def get_comparison(triplet):
    head = (df['latitude'][triplet[0]], df['longitude'][triplet[0]])
    left = (df['latitude'][triplet[1]], df['longitude'][triplet[1]])
    right = (df['latitude'][triplet[2]], df['longitude'][triplet[2]])
    return geodesic(head, left).miles < geodesic(head, right).miles

In [137]:
def get_true_embedding(df):
    X = np.zeros([len(df),2])
    
    for i in range(len(df)):
        X[i,0] = df['latitude'][i]
        X[i,1] = df['longitude'][i]
    return X

In [191]:
def generate_samples(sample_size, num_points, df):
    '''
    Generate triplet data
    '''
    # X real embedding
    triplets = []
    labels = []
    for _ in range(sample_size):
        triplet = generate_random_triplet(num_points)
        triplets.append(triplet)
        if get_comparison(triplet):
            labels.append([0,1])
        else:
            labels.append([1,0])

    return get_true_embedding(df), {'samples': triplets, 'labels':labels}, 

def generate_samples(sample_size, num_points, df):
    '''
    Generate triplet data
    '''
    # X real embedding
    triplets = []
    labels = []
    for i in range(num_points):
        print(i)
        for j in range(num_points):
            for k in range(num_points):
                if i!=j and i!=k and j!=k:
                    q = [i , j, k]
                    if get_comparison(q):
                        labels.append([0,1])
                    else:
                        labels.append([1,0])
                    triplets.append(q)
    random_indices = np.random.choice(int(num_points*(num_points-1)*(num_points-2)/2), sample_size, replace=False)
    return get_true_embedding(df), {'samples': [triplets[i] for i in random_indices], 'labels':[labels[i] for i in random_indices]}, {'samples': triplets, 'labels':labels}




def procrustes(X, Y, scaling=True, reflection='best'):
    n = X.shape[0]; m = X.shape[1]
    ny = Y.shape[0]; my = Y.shape[1]
    muX = X.mean(0); muY = Y.mean(0)
    X0 = X - muX; Y0 = Y - muY
    ssX = (X0**2.).sum(); ssY = (Y0**2.).sum()
    normX = np.sqrt(ssX); normY = np.sqrt(ssY)
    X0 /= normX; Y0 /= normY
    if my < m:
        Y0 = np.concatenate((Y0, np.zeros(n, m-my)),0)
    A = np.dot(X0.T, Y0)
    U,s,Vt = np.linalg.svd(A,full_matrices=False)
    V = Vt.T
    T = np.dot(V, U.T)
    if reflection is not 'best':
        have_reflection = np.linalg.det(T) < 0
        if reflection != have_reflection:
            V[:,-1] *= -1
            s[-1] *= -1
            T = np.dot(V, U.T)
    traceTA = s.sum()
    if scaling:
        b = traceTA * normX / normY
        d = 1 - traceTA**2
        Z = normX*traceTA*np.dot(Y0, T) + muX
    else:
        b = 1
        d = 1 + ssY/ssX - 2 * traceTA * normY / normX
        Z = normY*np.dot(Y0, T) + muX
    if my < m:
        T = T[:my,:]
    c = muX - b*np.dot(muY, T)
    tform = {'rotation':T, 'scale':b, 'translation':c}
    return d, Z, tform

In [192]:
def find_embedding(df, sample_size, n, decay_rate, initial_scale, initial_step, batch_size):
    decay = str(decay_rate)
    items = str(n)
    init_scale = str(initial_scale)
    init_step = str(initial_step)

    data = []
    d=2
    # Build the triples
    X, train, entire = generate_samples(sample_size, n, df)

    print('num constraints', len(train['samples']))
    train_size = len(train['samples'])

    # Build the Triplet Net
    # W - rows of W will be our learned embedding
    W = tf.Variable(tf.random_normal([n, d], 0., initial_scale), name="weights")
    head = tf.placeholder(tf.float32, [None, n])
    left = tf.placeholder(tf.float32, [None, n])
    right = tf.placeholder(tf.float32, [None, n])
    y = tf.placeholder(tf.float32, [None, 2])

    # Computes |W*e_i - W*e_j|^2
    dleft = tf.pow(tf.norm(tf.subtract(tf.matmul(head,W), tf.matmul(left, W)), axis=1), 2.)
    # Computes |W*e_i - W*e_k|^2
    dright = tf.pow(tf.norm(tf.subtract(tf.matmul(head,W), tf.matmul(right, W)), axis=1), 2.)

    # Hinge loss variant
    p = tf.stack([dleft-dright, dright-dleft], axis=1)

    loss = tf.reduce_mean(tf.maximum(0., 1.-tf.reduce_sum(y*p, reduction_indices=[1])))#tf.losses.hinge_loss(y, p)

    I = np.eye(n,n)
    head_data = [I[q[0], :] for q in train['samples']]
    left_data = [I[q[1], :] for q in train['samples']]
    right_data = [I[q[2], :] for q in train['samples']]
    
    head_data_entire = [I[q[0], :] for q in entire['samples']]
    left_data_entire = [I[q[1], :] for q in entire['samples']]
    right_data_entire = [I[q[2], :] for q in entire['samples']]

    #shuffle the data
    indices = np.random.permutation(train_size)

    # For debugging purposes
    #from tensorflow.python import debug as tf_debug
    sess = tf.InteractiveSession()

    # Setup the optimizer and pass in the loss function/all the data batched in sets of 100 for SGD
    global_step = tf.Variable(1.0, trainable=False)
    learning_rate = tf.train.exponential_decay(initial_step, global_step, len(head_data), decay_rate)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    var_grad = tf.gradients(loss, W)[0]

    # Compute training/test loss
    correct_prediction = tf.equal(tf.argmax(p,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    tf.global_variables_initializer().run()

    initials = sess.run([W, loss, y*p, 1.-tf.reduce_sum(y*p, reduction_indices=[1])],feed_dict={head: head_data,
                                                              left: left_data,
                                                              right: right_data,
                                                      y: train['labels']})
    Winitial = initials[0]
    start = time.time()
    iteration = 0
    old_loss = 0
    
    for _ in range(2000):
        print('iter {}'.format(_))
        x = sess.run([accuracy, loss, var_grad, learning_rate],
                              feed_dict={head: head_data,
                                         left: left_data,
                                         right: right_data,
                                         y: train['labels']})

        print('accuracy', x[0], 'loss', x[1], 'gradient', np.linalg.norm(x[2],ord='fro'), 'learning_rate', x[3])

        iteration += 1

        if x[0] == 1 or old_loss == x[1] :
            break
        old_loss = x[1]

        for i in range(1, int(np.round(train_size/batch_size))):
            if len(head_data[batch_size*(i-1): batch_size*i]) > 0:
                sess.run([train_step],
                          feed_dict={head: [head_data[j] for j in indices[batch_size*(i-1): batch_size*i]],
                                     left: [left_data[j] for j in indices[batch_size*(i-1): batch_size*i]],
                                     right: [right_data[j] for j in indices[batch_size*(i-1): batch_size*i]],
                                     y: [train['labels'][j] for j in indices[batch_size*(i-1): batch_size*i]]})
            else:
                sess.run([train_step],
                          feed_dict={head: [head_data[j] for j in indices[batch_size*(i-1):]],
                                     left: [left_data[j] for j in indices[batch_size*(i-1): ]],
                                     right: [right_data[j] for j in indices[batch_size*(i-1):]],
                                     y: [train['labels'][j] for j in indices[batch_size*(i-1):]]})
        indices = np.random.permutation(train_size)

    result_test = sess.run([accuracy, W, loss,p], feed_dict={head: head_data,
                                                     left: left_data,
                                                     right: right_data,
                                                     y: train['labels']})
    
    result_entire = sess.run([accuracy, W, loss,p], feed_dict={head: head_data_entire,
                                                     left: left_data_entire,
                                                     right: right_data_entire,
                                                     y: entire['labels']})
    end = time.time()
    total_time=(end - start)

    print('Final loss', result_test[2])
    print('Accuracy on train set:', result_test[0], result_test[1])
    print('Accuracy on entire data:', result_entire[0], result_entire[1])
    min_gap = np.min(np.abs(result_test[3]))
    print('min gap', min_gap)

    #Procrustes and plot
    if d==2:
        W = result_test[1]
        _, Wpro, _ = procrustes(X, W)
#         plt.scatter(Wpro[:,0], Wpro[:,1])
#         plt.scatter(X[:,0], X[:,1])
        
        for i in range(len(df)):
            x = 10*X[i,1]
            y = 10*X[i,0]
            plt.plot(x, y, 'bo')
            plt.text(x * (1 + 0.01), y * (1 + 0.01) , df['name'][i], fontsize=12)
        plt.show()
        for i in range(len(df)):
            x = 10*Wpro[i,1]
            y = 10*Wpro[i,0]
            plt.plot(x, y, 'bo')
            plt.text(x * (1 + 0.01), y * (1 + 0.01) , df['name'][i], fontsize=12)
            
        plt.show()
        
        
        frob_error = np.linalg.norm(X-Wpro,'fro')

In [ ]:
#find_embedding(df, sample_size, n, decay_rate, initial_scale, initial_step, batch_size)
#find_embedding(df, 3000, 48, .09, 1, .9, 5)
find_embedding(df, 5000, 50, .09, 1, .9, 5)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
num constraints 5000
iter 0


/anaconda/envs/tensorflow/lib/python3.5/site-packages/numpy/linalg/linalg.py:2176: RuntimeWarning: invalid value encountered in sqrt
  ret = sqrt(sqnorm)


accuracy 0.4926 loss 2.19236 gradient nan learning_rate 0.899567
iter 1
accuracy 0.8294 loss 0.690523 gradient nan learning_rate 0.556021
iter 2
accuracy 0.8932 loss 0.336481 gradient nan learning_rate 0.343676
iter 3
accuracy 0.9446 loss 0.174823 gradient 1.41421 learning_rate 0.212425
iter 4
accuracy 0.9624 loss 0.116595 gradient nan learning_rate 0.1313
iter 5
accuracy 0.98 loss 0.0680362 gradient nan learning_rate 0.0811562
iter 6
accuracy 0.9844 loss 0.0538571 gradient nan learning_rate 0.0501625
iter 7
accuracy 0.9894 loss 0.0432221 gradient 0.0 learning_rate 0.0310054
iter 8
accuracy 0.9936 loss 0.038419 gradient 1.41421 learning_rate 0.0191644
iter 9
accuracy 0.9932 loss 0.0355083 gradient 0.0 learning_rate 0.0118455
iter 10
accuracy 0.9934 loss 0.0339005 gradient 6.074e+09 learning_rate 0.00732167
iter 11
accuracy 0.994 loss 0.0336563 gradient nan learning_rate 0.00452551
iter 12
accuracy 0.9938 loss 0.0331261 gradient 3.29272e-10 learning_rate 0.00279721
iter 13
accuracy 0.99

In [79]:
df

,name,description,latitude,longitude
0,Alabama,Montgomery,32.377716,-86.300568
1,Arizona,Phoenix,33.448143,-112.096962
2,Arkansas,Little Rock,34.746613,-92.288986
3,California,Sacramento,38.576668,-121.493629
4,Colorado,Denver,39.739227,-104.984856
5,Connecticut,Hartford,41.764046,-72.682198
6,Delaware,Dover,39.157307,-75.519722
7,Florida,Tallahassee,30.438118,-84.281296
8,Georgia,Atlanta,33.749027,-84.388229
9,Idaho,Boise,43.617775,-116.199722
